# <font color='purple'> DANG Méline - Exercice Kick Starter </font>


#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [2]:
import pandas as pd
import pymongo
import json

In [3]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['kickstarter']

In [4]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [5]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [6]:
print(df_ks.dtypes)

ID                    int64
name                 object
category             object
main_category        object
currency             object
deadline             object
goal                 object
launched             object
pledged              object
state                object
backers              object
country              object
usd pledged          object
usd_pledged_real    float64
dtype: object


In [7]:
# Taille de notre jeu de données
print(df_ks.shape)
# Nous regardons combien nous avons de valeur NaN par colonnes
print(df_ks.isna().sum())

(150000, 14)
ID                     0
name                   2
category               0
main_category          0
currency               0
deadline               0
goal                   0
launched               0
pledged                0
state                  0
backers                0
country                0
usd pledged         1482
usd_pledged_real       0
dtype: int64


In [8]:
# Certaines colonnes telles que name et usd pledge comportent des données NaN, nous supprimons donc les lignes correspondantes.
# Il n'est pas nécessaire de les supprimer puisque mongo ne prendra pas de mémoire pour ces données.
indexNaN = df_ks[ df_ks['name'].isna() | df_ks['usd pledged'].isna() ].index
df_ks.drop(indexNaN , inplace=True)
print(df_ks.isna().sum())

ID                  0
name                0
category            0
main_category       0
currency            0
deadline            0
goal                0
launched            0
pledged             0
state               0
backers             0
country             0
usd pledged         0
usd_pledged_real    0
dtype: int64


In [9]:
# Nous supprimons les colonnes qui ne sont pas utiles pour répondre aux questions.
df_ks = df_ks.drop(columns=["backers", "goal"])

### Importer les données

In [10]:
df = df_ks.to_json(orient="records")

In [11]:
df = json.loads(df)

In [ ]:
collection.insert_many(df)

In [ ]:
print(type(collection))

## Question 1  
Récupérer les 5 projets ayant reçu le plus de promesse de dons.

In [ ]:
collection.find_one()

In [ ]:
# Récupération des 5 projets ayant reçu le plus de promesse de dons.
# Nous trions par usd pledged, et prenons les 5 premiers.
cur = collection.find().sort("usd pledged",1).limit(5)
list(cur) 

## Question 2
Compter le nombre de projets ayant atteint leur but.

In [ ]:
# Projets ayant atteint leur but.
cur = collection.find({"state": "successful"})
list(cur)

In [ ]:
# Nombre de projets ayant atteint leur but.
# Nous sélectionnons uniquement les projets ayant atteint leur but (state : successful).
cur = collection.aggregate([
    {"$match":{"state":"successful"}},
    {"$group" : {"_id" : "$state", "Nombre de projets ayant atteint leur but" : {"$sum" : 1}}}
    ])
list(cur)

## Question 3
Compter le nombre de projets pour chaque catégorie.

In [ ]:
# Nombre de projets pour chaque catégorie :
# Nous regroupons par catégorie et nous comptons.
cur = collection.aggregate([
    {"$group" : {"_id" : "$category", "Nombre de projets pour cette catégorie" : {"$sum" : 1}}}
    ])
list(cur)

In [ ]:
# Nombre de projets pour chaque catégorie principale :
# Nous regroupons par main catégorie et nous comptons.
cur = collection.aggregate([
    {"$group" : {"_id" : "$main_category", "Nombre de projets pour cette catégorie" : {"$sum" : 1}}}
    ])
list(cur)

## Question 4
Compter le nombre de projets français ayant été instanciés avant 2016.

In [ ]:
# Nombre de projets français ayant été instanciés avant 2016.
# Nous sélectionnons uniquement les projets de la France (country : France)
# ainsi que leur année d'instanciation avant 2016 (launched < 2016).
# Nous comptons ensuite le tout.
                
                
cur = collection.aggregate([
    {"$match": { "$and": [ {"launched": { "$lt": "2016-01-01"} }, { "country" : "FR" } ] } },
    {"$group" : {"_id" : "null", "Nombre de projets français ayant été instanciés avant 2016" : {"$sum" : 1}}}
    ])
list(cur)  

In [ ]:
cur = collection.find({"country":"FR",
                       "launched":{"$lt":"2016-01-01"}}
                     ).count()
cur

## Question 5
Récupérer les projets américains ayant demandé plus de 200 000 dollars.

In [ ]:
# Nombre de projets américains ayant demandé plus de 200 000 dollars.
# Nous sélectionnons uniquement les projets des Etats-Unis (country : US)
# ainsi que les projets ayant demandé plus de 200 000 dollars (usd pledged > 200 000).
# Nous comptons ensuite le tout.
cur = collection.aggregate([
    {"$match": { "$and": [ {"usd pledged": { "$gt": 200000} }, { "country" : "US" } ] } },
    {"$group" : {"_id" : "null", "Nombre de projets américains ayant demandé plus de 200 000 dollars" : {"$sum" : 1}}}
    ])
list(cur)  
    

## Question 6
Compter le nombre de projet ayant "Sport" dans leur nom

In [ ]:
# Nombre de projet ayant "Sport" dans leur nom.
# Nous sélectionnons les projets ayant "Sport" dans leur nom avec regex puis non les comptons.
cur = collection.aggregate([
    {"$match" : {"name" : {"$regex" : "Sport"}}},
    {"$group" : {"_id" : "null", "Nombre de projets ayant 'Sport' dans leur nom" : {"$sum" : 1}}}
    ])
list(cur)

In [ ]:
# Exemple d'un projet contenant "Sport" dans son nom.
cur = collection.find({"name" : {"$regex" : "Sport"}});
next(cur)

In [ ]:
cur = collection.find({"name" : {"$regex" : "Sport" }}).count()
cur